## Workshop นี้ดึงข้อมูลจาก Database และ REST API 

## อ่านข้อมูลจาก MySQL Database

In [1]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.7 MB/s 


config สำหรับเชื่อต่อ MySQL database

In [2]:
# สำหรับ Config เชื่อมต่อกับ MySQL มาจากคอร์ส Data TH - Data Science ชิลชิล
# ขอทำการซ่อนรหัสผ่านเพื่อนป้องกัน แต่ Config นี้สามารถใช้งานได้
class Config :
    MYSQL_HOST = ''
    MYSQL_PORT = 
    MYSQL_USER = ''
    MYSQL_PASSWORD = ''
    MYSQL_DB = ''
    MYSQL_CHARSET = ''

In [3]:
# ทดลอง print ใน Config 
print(Config.MYSQL_PORT)

3306


## Connect to DB

In [4]:
import pymysql

# Connect to database
connection = pymysql.connect(host=Config.MYSQL_HOST,
                             port=Config.MYSQL_PORT,
                             user=Config.MYSQL_USER,
                             password=Config.MYSQL_PASSWORD,
                             db=Config.MYSQL_DB,
                             charset=Config.MYSQL_CHARSET,
                             cursorclass=pymysql.cursors.DictCursor)

In [5]:
connection

## List Tables 
เช็ค Table มีอะไรบ้างใน list

In [6]:
cursor = connection.cursor()
cursor.execute('show tables')
tables = cursor.fetchall()
cursor.close()
print(tables)

[{'Tables_in_r2de2': 'audible_data'}, {'Tables_in_r2de2': 'audible_transaction'}]


## Query Table
ใช้ with statment แทน cursor.close()

In [7]:
# table ที่ดึงจาก Database มีตาราง audible_data, audible_transaction

with connection.cursor() as cursor:
    cursor.execute('SELECT * FROM audible_data')
    result_audible_data = cursor.fetchall()

print('Len audible_data', len(result_audible_data))

Len audible_data 2269


In [8]:
with connection.cursor() as cursor:
    cursor.execute('SELECT * FROM audible_transaction')
    result_audible_transaction = cursor.fetchall()

print('Len audible_transaction', len(result_audible_transaction))

Len audible_transaction 1998821


In [9]:
# เช็คในตัวแปร audible_data เป็นชนิดไหน
type(result_audible_data)

list

In [10]:
# ลอง print ว่า result_audible_data ข้อมูลอยู่ใน list 
result_audible_data

[{'Audio Runtime': '6 hrs and 31 mins',
  'Audiobook_Type': 'Unabridged Audiobook',
  'Book Author': 'Yvonne Orji',
  'Book Narrator': 'Yvonne Orji',
  'Book Subtitle': 'How God Tricked Me into the Life of My Dreams',
  'Book Title': 'Bamboozled by Jesus',
  'Book_ID': 1,
  'Categories': 'Biographies & Memoirs',
  'Price': '$29.65 ',
  'Rating': '5',
  'Total No. of Ratings': 47},
 {'Audio Runtime': '13 hrs and 33 mins',
  'Audiobook_Type': 'Unabridged Audiobook',
  'Book Author': 'Michael Chatfield',
  'Book Narrator': 'Neil Hellegers',
  'Book Subtitle': 'A LitRPG Fantasy Series (The Ten Realms, Book 6)',
  'Book Title': 'Sixth Realm Part 1',
  'Book_ID': 2,
  'Categories': 'Science Fiction & Fantasy',
  'Price': '$24.95 ',
  'Rating': '4.5',
  'Total No. of Ratings': 98},
 {'Audio Runtime': '27 hrs and 30 mins',
  'Audiobook_Type': 'Unabridged Audiobook',
  'Book Author': 'Diana Gabaldon',
  'Book Narrator': 'Davina Porter',
  'Book Subtitle': 'Outlander, Book 9',
  'Book Title': 'G

## Convert Pandas 
แปลงเป็น DataFrame เพื่อดูข้อมูลง่ายขึ้น

In [11]:
import pandas as pd

audible_data = pd.DataFrame(result_audible_data)
audible_transaction = pd.DataFrame(result_audible_transaction)

In [12]:
audible_data.head()

,Book_ID,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
1,2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
2,3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
3,4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
4,5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65


In [13]:
audible_transaction.head()

,timestamp,user_id,book_id,country
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal
1,2021-05-01 00:00:03,561b26c1,829,United States of America
2,2021-05-01 00:00:04,81f149e5,1391,Japan
3,2021-05-01 00:00:07,4f218413,1586,Taiwan
4,2021-05-01 00:00:18,a4066781,300,United States of America


In [14]:
# เช็ค Type audible_data
type(audible_data)

pandas.core.frame.DataFrame

ใน audible_data มี Index ด้านหน้า แต่ในตาราง audible_data มีคอลัมน์ Book_ID ที่เป็นตัวเลข unique อยู่แล้ว จะกำหนด Book_ID เป็น Index

In [15]:
audible_data = audible_data.set_index('Book_ID')
audible_data.head()

,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
Book_ID,,,,,,,,,,
1,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5,47.0,$29.65
2,Sixth Realm Part 1,"A LitRPG Fantasy Series (The Ten Realms, Book 6)",Michael Chatfield,Neil Hellegers,13 hrs and 33 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.5,98.0,$24.95
3,Go Tell the Bees That I Am Gone,"Outlander, Book 9",Diana Gabaldon,Davina Porter,27 hrs and 30 mins,Unabridged Audiobook,Science Fiction & Fantasy,None,NaN,$41.99
4,How the Word Is Passed,A Reckoning with the History of Slavery Across...,Clint Smith,Clint Smith,10 hrs and 7 mins,Unabridged Audiobook,History,None,NaN,$29.65
5,The Devil May Dance,A Novel,Jake Tapper,Rob Shapiro,10 hrs and 12 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.5,108.0,$29.65


## สามารถ Query โดยใช้ Pandas ใช้ฟังก์ชัน read_sql()

In [16]:
sql = 'SELECT * FROM audible_transaction'
df_audible_transaction = pd.read_sql(sql, connection)
df_audible_transaction.head()

,timestamp,user_id,book_id,country
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal
1,2021-05-01 00:00:03,561b26c1,829,United States of America
2,2021-05-01 00:00:04,81f149e5,1391,Japan
3,2021-05-01 00:00:07,4f218413,1586,Taiwan
4,2021-05-01 00:00:18,a4066781,300,United States of America


## Join Table 

ทำการ join ตาราง audible_data & audible_transaction

ใน audible_transaction ไม่มีข้อมูลชื่อสินค้า อยากรู้ว่า audible_transaction มีราคาเท่าไร ต้อง merge เข้ากับ audible_data

In [17]:
transaction = audible_transaction.merge(audible_data,
                                        how='left',
                                        left_on='book_id',
                                        right_on='Book_ID')

In [18]:
transaction.head()

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49


ได้ข้อมูล transaction แต่จะแปลงข้อมูล price จาก USD เป็น THB ต่อไปจะอานข้อมูลจาก API เพื่อแปลงค่าเงินบาท

Get Data From REST API

In [19]:
import requests

In [20]:
# อ่านข้อมูลจาก API
url = 'https://r2de2-workshop-vmftiryt6q-ts.a.run.app/usd_thb_conversion_rate'
r = requests.get(url)
print(r.status_code)
result_conversion_rate = r.json()

200


In [21]:
result_conversion_rate

{'conversion_rate': {'2021-04-01': 31.194,
  '2021-04-02': 31.29,
  '2021-04-03': 31.256,
  '2021-04-04': 31.244,
  '2021-04-05': 31.342,
  '2021-04-06': 31.288,
  '2021-04-07': 31.387,
  '2021-04-08': 31.433,
  '2021-04-09': 31.451,
  '2021-04-10': 31.454,
  '2021-04-11': 31.462,
  '2021-04-12': 31.482,
  '2021-04-13': 31.496,
  '2021-04-14': 31.329,
  '2021-04-15': 31.165,
  '2021-04-16': 31.203,
  '2021-04-17': 31.204,
  '2021-04-18': 31.226,
  '2021-04-19': 31.203,
  '2021-04-20': 31.281,
  '2021-04-21': 31.3,
  '2021-04-22': 31.378,
  '2021-04-23': 31.412,
  '2021-04-24': 31.411,
  '2021-04-25': 31.421,
  '2021-04-26': 31.427,
  '2021-04-27': 31.355,
  '2021-04-28': 31.344,
  '2021-04-29': 31.197,
  '2021-04-30': 31.139,
  '2021-05-01': 31.14,
  '2021-05-02': 31.122,
  '2021-05-03': 31.132,
  '2021-05-04': 31.188,
  '2021-05-05': 31.13,
  '2021-05-06': 31.226,
  '2021-05-07': 31.042,
  '2021-05-08': 31.042,
  '2021-05-09': 31.033,
  '2021-05-10': 31.107,
  '2021-05-11': 31.158,
  

In [22]:
print(type(result_conversion_rate))

<class 'dict'>


## Convert result_conversion_rate to Pandas

In [23]:
conversion_rate = pd.DataFrame(result_conversion_rate)
conversion_rate.head()

,conversion_rate
2021-04-01,31.194
2021-04-02,31.290
2021-04-03,31.256
2021-04-04,31.244
2021-04-05,31.342


In [24]:
print(type(conversion_rate))

<class 'pandas.core.frame.DataFrame'>


In [25]:
conversion_rate.info()

<class 'pandas.core.frame.DataFrame'>
Index: 134 entries, 2021-04-01 to 2021-08-12
Data columns (total 1 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   conversion_rate  134 non-null    float64
dtypes: float64(1)
memory usage: 2.1+ KB


ทำการ set index ของ table conversion_rate และเปลี่ยนช่ื่อคอลัมน์ เพราะทำการ join กับ transaction

In [26]:
conversion_rate = conversion_rate.reset_index().rename(columns={'index': 'date'})

In [27]:
conversion_rate[:3]

,date,conversion_rate
0,2021-04-01,31.194
1,2021-04-02,31.290
2,2021-04-03,31.256


## Join Data conversion_rate & transaction

เรามีข้อมูลการซื้อขายและข้อมูล Rage แปลงค่าเงิน รวม Dataframe ทั้ง 2 มารวมกันผ่าน column ชื่อว่า date ใน transaction และ conversion_rate

In [28]:
# สร้างคอลัมน์ใหม่ transaction ชื่อว่า date โดย copy คอลัมน์ timestamp เพื่อทำการ join
transaction['date'] = transaction['timestamp']
transaction.columns

Index(['timestamp', 'user_id', 'book_id', 'country', 'Book Title',
       'Book Subtitle', 'Book Author', 'Book Narrator', 'Audio Runtime',
       'Audiobook_Type', 'Categories', 'Rating', 'Total No. of Ratings',
       'Price', 'date'],
      dtype='object')

In [29]:
# แปลงให้จาก timestamp เป็น date ในทั้ง 2 dataframe (transaction, conversion_rate)
transaction['date'] = pd.to_datetime(transaction['date']).dt.date
conversion_rate['date'] = pd.to_datetime(conversion_rate['date']).dt.date

In [30]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998821 entries, 0 to 1998820
Data columns (total 15 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   user_id               object        
 2   book_id               int64         
 3   country               object        
 4   Book Title            object        
 5   Book Subtitle         object        
 6   Book Author           object        
 7   Book Narrator         object        
 8   Audio Runtime         object        
 9   Audiobook_Type        object        
 10  Categories            object        
 11  Rating                object        
 12  Total No. of Ratings  float64       
 13  Price                 object        
 14  date                  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(12)
memory usage: 244.0+ MB


In [31]:
conversion_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             134 non-null    object 
 1   conversion_rate  134 non-null    float64
dtypes: float64(1), object(1)
memory usage: 2.2+ KB


In [32]:
# join 
final_df = transaction.merge(conversion_rate,
                             how='left',
                             left_on='date',
                             right_on='date')

In [33]:
final_df.head()

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,date,conversion_rate
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,$35.00,2021-05-01,31.14
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,$19.59,2021-05-01,31.14
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,$31.50,2021-05-01,31.14
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,$19.95,2021-05-01,31.14
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,$17.49,2021-05-01,31.14


In [34]:
final_df.columns

Index(['timestamp', 'user_id', 'book_id', 'country', 'Book Title',
       'Book Subtitle', 'Book Author', 'Book Narrator', 'Audio Runtime',
       'Audiobook_Type', 'Categories', 'Rating', 'Total No. of Ratings',
       'Price', 'date', 'conversion_rate'],
      dtype='object')

In [35]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998821 entries, 0 to 1998820
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   user_id               object        
 2   book_id               int64         
 3   country               object        
 4   Book Title            object        
 5   Book Subtitle         object        
 6   Book Author           object        
 7   Book Narrator         object        
 8   Audio Runtime         object        
 9   Audiobook_Type        object        
 10  Categories            object        
 11  Rating                object        
 12  Total No. of Ratings  float64       
 13  Price                 object        
 14  date                  object        
 15  conversion_rate       float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(12)
memory usage: 259.2+ MB


คอลัมน์ Price มีเครื่องหมาย $ และเป็น Type String ทำความสะอาดข้อมูลและเปลี่ยน Type เป็น float ใช้ function lambda สำหรับประมวลผลแต่ละแถว

In [36]:
final_df['Price'] = final_df.apply(lambda x: x['Price'].replace('$',''), axis=1)
final_df.head(1)

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,date,conversion_rate
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,2021-05-01,31.14


In [37]:
# แปลง Type คอลัมน์ Price และ Rating ให้เป็นตัวเลข
final_df['Price'] = final_df['Price'].astype('float')

In [38]:
# แปลง Rating เป็น int
final_df['Rating'] = pd.to_numeric(final_df['Rating'],errors='coerce')

In [39]:
final_df.dtypes

timestamp               datetime64[ns]
user_id                         object
book_id                          int64
country                         object
Book Title                      object
Book Subtitle                   object
Book Author                     object
Book Narrator                   object
Audio Runtime                   object
Audiobook_Type                  object
Categories                      object
Rating                         float64
Total No. of Ratings           float64
Price                          float64
date                            object
conversion_rate                float64
dtype: object

ทำการ join ข้อมูลได้แล้ว สร้างคอลัมน์ใหม่ว่า THBPrice แล้วนำคอลัมน์ Price * conversion_rate 

In [40]:
final_df['THBPrice'] = final_df['Price'] * final_df['conversion_rate']
final_df.head(3)

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,date,conversion_rate,THBPrice
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,2021-05-01,31.14,1089.9000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,2021-05-01,31.14,610.0326
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,2021-05-01,31.14,980.9100


อีกวิธีหนึ่งสามารถ สร้าง function คำนวนราคาบาท ได้ผลลัพธ๊เหมือนกัน

In [41]:
final_df2 = final_df

In [42]:
def convert_price(price, rate):
    return price * rate

final_df2['THBPrice'] = final_df2.apply(lambda x: convert_price(x['Price'], x['conversion_rate']), axis=1)
final_df2.head(3)

,timestamp,user_id,book_id,country,Book Title,Book Subtitle,Book Author,Book Narrator,Audio Runtime,Audiobook_Type,Categories,Rating,Total No. of Ratings,Price,date,conversion_rate,THBPrice
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,2021-05-01,31.14,1089.9000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,2021-05-01,31.14,610.0326
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,2021-05-01,31.14,980.9100


final_df มีคอลัมน์ที่ซ้ำกันสามารถลบออกได้ คอลัมน์ที่ซ้ำคือ date ที่ซ้ำซ้อนกับ timestamp ลบคอลัมน์ date เพราะข้อมูลเหมือนกัน

In [43]:
final_df = final_df.drop('date', axis=1)

In [44]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1998821 entries, 0 to 1998820
Data columns (total 16 columns):
 #   Column                Dtype         
---  ------                -----         
 0   timestamp             datetime64[ns]
 1   user_id               object        
 2   book_id               int64         
 3   country               object        
 4   Book Title            object        
 5   Book Subtitle         object        
 6   Book Author           object        
 7   Book Narrator         object        
 8   Audio Runtime         object        
 9   Audiobook_Type        object        
 10  Categories            object        
 11  Rating                float64       
 12  Total No. of Ratings  float64       
 13  Price                 float64       
 14  conversion_rate       float64       
 15  THBPrice              float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(9)
memory usage: 259.2+ MB


## Save CSV

In [45]:
final_df.to_csv('output.csv', index=False)

## Basic Pandas

In [46]:
final_df.shape

(1998821, 16)

เปลี่ยนชื่อคอลัมน์

In [47]:
final_df.columns

Index(['timestamp', 'user_id', 'book_id', 'country', 'Book Title',
       'Book Subtitle', 'Book Author', 'Book Narrator', 'Audio Runtime',
       'Audiobook_Type', 'Categories', 'Rating', 'Total No. of Ratings',
       'Price', 'conversion_rate', 'THBPrice'],
      dtype='object')

In [48]:
final_df.rename(columns={'Book Title':'Book_Title',
                         'Book Subtitle':'Book_Subtitle',
                         'Book Author':'Book_Author',
                         'Book Narrator':'Book_Narrator',
                         'Audio Runtime':'Audio_Runtime',
                         'Total No. of Ratings':'Total_No_of_Ratings'},
                         inplace=True)
final_df.columns

Index(['timestamp', 'user_id', 'book_id', 'country', 'Book_Title',
       'Book_Subtitle', 'Book_Author', 'Book_Narrator', 'Audio_Runtime',
       'Audiobook_Type', 'Categories', 'Rating', 'Total_No_of_Ratings',
       'Price', 'conversion_rate', 'THBPrice'],
      dtype='object')

เลือกคอลัมน์ที่ต้องการดู 

In [49]:
final_df[['book_id','country','Book_Title','Categories']]

,book_id,country,Book_Title,Categories
0,1584,Portugal,Sycamore Row,"Mystery, Thriller & Suspense"
1,829,United States of America,The 5 Love Languages of Children,"Relationships, Parenting & Personal Development"
2,1391,Japan,The Guardians,"Mystery, Thriller & Suspense"
3,1586,Taiwan,How to Be a 3% Man,"Relationships, Parenting & Personal Development"
4,300,United States of America,Napoleon Hill's Outwitting the Devil,Business & Careers
...,...,...,...,...
1998816,600,Portugal,The Time of Contempt,Science Fiction & Fantasy
1998817,462,Montenegro,Endurance,History
1998818,1656,United States of America,The Spy and the Traitor,Biographies & Memoirs
1998819,522,United States of America,Lilac Girls,Literature & Fiction


In [50]:
final_df[0:5][['book_id','country','Book_Title','Categories']]

,book_id,country,Book_Title,Categories
0,1584,Portugal,Sycamore Row,"Mystery, Thriller & Suspense"
1,829,United States of America,The 5 Love Languages of Children,"Relationships, Parenting & Personal Development"
2,1391,Japan,The Guardians,"Mystery, Thriller & Suspense"
3,1586,Taiwan,How to Be a 3% Man,"Relationships, Parenting & Personal Development"
4,300,United States of America,Napoleon Hill's Outwitting the Devil,Business & Careers


groupby จัดกลุ่มข้อมูล

In [51]:
# จัดกลุ่ม Categories ของหนังสือทั้งหมดที่มี
categories_book = final_df['Categories'].value_counts()
categories_book

Literature & Fiction                               547364
Science Fiction & Fantasy                          326089
Biographies & Memoirs                              197481
Business & Careers                                 156190
Children's Audiobooks                              141040
Relationships, Parenting & Personal Development    115570
Mystery, Thriller & Suspense                       111706
Health & Wellness                                   94204
Romance                                             56693
Teen                                                54658
Arts & Entertainment                                38918
Politics & Social Sciences                          33767
History                                             33314
Money & Finance                                     27180
Religion & Spirituality                             11724
Science & Engineering                                8425
LGBTQ+                                               5911
Education & Le

In [52]:
# รายชื่อประเทศของหนังสือทั้งหมด
country_book= final_df['country'].value_counts()
country_book

United States of America    745010
United Kingdom              209975
Canada                      124733
Australia                    51331
Germany                      20281
Jamaica                      20280
Greece                       20270
Taiwan                       20205
Brazil                       20173
Netherlands                  20139
Hungary                      20130
Montenegro                   20098
Vietnam                      20092
Switzerland                  20089
Colombia                     20083
Denmark                      20072
South Africa                 20072
Belgium                      20067
New Zealand                  20066
Turkey                       20059
Ireland                      20047
Portugal                     20044
Singapore                    20043
Finland                      20040
Mexico                       20025
Austria                      20021
Serbia                       19998
Argentina                    19959
Sweden              

In [53]:
# อีกวิธีหนึ่งหารายื่อประเทศ
final_df.groupby(['country'])['country'].count().nlargest(10)

country
United States of America    745010
United Kingdom              209975
Canada                      124733
Australia                    51331
Germany                      20281
Jamaica                      20280
Greece                       20270
Taiwan                       20205
Brazil                       20173
Netherlands                  20139
Name: country, dtype: int64

Filter และ Query ข้อมูลด้วยเงื่อนไข

In [68]:
# หาข้อมูลหนังสือที่มี Rating < 4 
final_df[final_df['Rating'] <= 4.0 ]

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
28,2021-05-01 00:01:37,0eb19649,2031,United States of America,Summary: Cal Newport's Deep Work: Rules for Fo...,None,Ant Hive Media,Doron Alon,17 mins,Unabridged Audiobook,None,3.9,70.0,3.95,31.140,123.00300
33,2021-05-01 00:01:57,27fbb0fe,1734,Argentina,10 Essential Pieces of Literature,None,Robert Louis Stevenson,Matthew Taylor,87 hrs and 44 mins,Unabridged Audiobook,Literature & Fiction,3.3,85.0,14.95,31.140,465.54300
126,2021-05-01 00:08:31,1709fcf2,2099,Sweden,Summary of Shad Helmstetter's What to Say When...,None,Milkyway Media,Adrienne Walker,24 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",3.7,41.0,3.95,31.140,123.00300
250,2021-05-01 00:17:03,21ac7756,1418,Hungary,Fifty Shades of Grey,Book One of the Fifty Shades Trilogy,E. L. James,Becca Battoe,19 hrs and 48 mins,Unabridged Audiobook,Literature & Fiction,3.9,35537.0,33.60,31.140,1046.30400
283,2021-05-01 00:19:32,67f860d4,879,Algeria,We Were Liars,None,E. Lockhart,Ariadne Meyers,6 hrs and 27 mins,Unabridged Audiobook,Teen,4.0,4759.0,25.20,31.140,784.72800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997879,2021-07-31 23:00:45,eb871d17,1920,United States of America,In Charge of Moonlight,None,Dina Gregory,Matt Addis,5 mins,Unabridged Audiobook,Teen,4.0,79.0,0.00,32.887,0.00000
1997931,2021-07-31 23:05:01,7349ea15,1046,Italy,The Vanishing Point,A Novel,Elizabeth Brundage,Fajer Al-Kaisi,10 hrs and 24 mins,Unabridged Audiobook,Literature & Fiction,4.0,1.0,29.65,32.887,975.09955
1998097,2021-07-31 23:15:52,e8b8472e,1418,United States of America,Fifty Shades of Grey,Book One of the Fifty Shades Trilogy,E. L. James,Becca Battoe,19 hrs and 48 mins,Unabridged Audiobook,Literature & Fiction,3.9,35537.0,33.60,32.887,1105.00320
1998168,2021-07-31 23:19:42,cdcd0b4a,1418,Ireland,Fifty Shades of Grey,Book One of the Fifty Shades Trilogy,E. L. James,Becca Battoe,19 hrs and 48 mins,Unabridged Audiobook,Literature & Fiction,3.9,35537.0,33.60,32.887,1105.00320


In [55]:
# เลือกหนังสือที่อยู่ในหมวดหมู่ Science Fiction & Fantasy
final_df[final_df['Categories'] == "Science Fiction & Fantasy" ]

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
6,2021-05-01 00:00:22,f636754b,342,Canada,A Clash of Kings,"A Song of Ice and Fire, Book 2",George R.R. Martin,Roy Dotrice,37 hrs and 12 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.8,79031.0,63.00,31.140,1961.82000
7,2021-05-01 00:00:26,0eea4bbd,1600,United States of America,Exile,"Legend of Drizzt: Dark Elf Trilogy, Book 2",R. A. Salvatore,Victor Bevine,10 hrs and 42 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,9241.0,24.95,31.140,776.94300
10,2021-05-01 00:00:31,1d4e0721,630,Canada,The Final Empire,Mistborn Book 1,Brandon Sanderson,Michael Kramer,24 hrs and 39 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.8,65789.0,55.93,31.140,1741.66020
11,2021-05-01 00:00:34,9711795f,1561,France,Morning Star,Book III of the Red Rising Trilogy,Pierce Brown,Tim Gerard Reynolds,21 hrs and 50 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.8,36539.0,31.49,31.140,980.59860
14,2021-05-01 00:00:47,980adb4b,74,United Kingdom,Dune,None,Frank Herbert,Scott Brick,21 hrs and 2 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.6,73222.0,47.93,31.140,1492.54020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998785,2021-07-31 23:58:03,53105c8c,488,United States of America,All These Worlds,"Bobiverse, Book 3",Dennis E. Taylor,Ray Porter,7 hrs and 56 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.8,57198.0,29.95,32.887,984.96565
1998788,2021-07-31 23:58:17,bdbce6ec,1453,United Kingdom,The Alloy of Law,A Mistborn Novel,Brandon Sanderson,Michael Kramer,10 hrs and 48 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,28422.0,27.99,32.887,920.50713
1998798,2021-07-31 23:58:38,c88d5e79,1434,Canada,Winter's Heart,"Wheel of Time, Book 9",Robert Jordan,Kate Reading,24 hrs and 12 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,18258.0,48.99,32.887,1611.13413
1998814,2021-07-31 23:59:45,17591e8d,1704,Hungary,#1 in Customer Service,The Complete Adventures of Tom Stranger,Larry Correia,Adam Baldwin,8 hrs and 34 mins,Original Recording Audiobook,Science Fiction & Fantasy,4.7,2105.0,24.95,32.887,820.53065


In [56]:
# หายอดขายหนังสือแต่ละประเภท
final_df.groupby('Categories').sum()['THBPrice']

Categories
Arts & Entertainment                               3.216671e+07
Biographies & Memoirs                              1.905936e+08
Business & Careers                                 1.232234e+08
Children's Audiobooks                              1.477603e+08
Computers & Technology                             6.849015e+05
Education & Learning                               1.962106e+06
Erotica                                            2.789397e+06
Health & Wellness                                  8.194556e+07
History                                            3.211153e+07
Home & Garden                                      3.338330e+06
LGBTQ+                                             4.547200e+06
Literature & Fiction                               5.534005e+08
Money & Finance                                    2.276557e+07
Mystery, Thriller & Suspense                       1.073871e+08
Politics & Social Sciences                         2.977301e+07
Relationships, Parenting & Pe

In [57]:
#ยอดขายหนังสือที่อยู่ในหมวดหมู่ Science Fiction & Fantasy 
final_df[final_df['Categories'] == "Science Fiction & Fantasy" ].groupby('Book_Title').sum()['THBPrice']

Book_Title
#1 in Customer Service                          2.607743e+05
1/2986: Publisher's Pack 2                      5.438566e+04
A Blood of Kings                                9.964737e+04
A Clash of Kings                                1.987207e+07
A Dance with Dragons                            1.027695e+07
                                                    ...     
White Wasteland                                 4.986709e+04
Winter's Heart                                  3.604597e+06
Within the Sanctuary of Wings                   3.811272e+04
Words of Radiance                               1.517549e+07
¿Sueñan los androides con ovejas eléctricas?    3.578979e+04
Name: THBPrice, Length: 196, dtype: float64

Filter 2 เงื่อนไข

In [58]:
# หาหนังสือในประเทศ United States of America หรือหนังสือในหมวดหมู่ Arts & Entertainment
final_df[(final_df['country'] == "United States of America" ) & \
         (final_df['Categories'] == "Arts & Entertainment" )]

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
193,2021-05-01 00:13:10,04677dc6,1427,United States of America,Open Book,A Memoir,Jessica Simpson,Jessica Simpson,11 hrs and 22 mins,Unabridged Audiobook,Arts & Entertainment,4.8,10158.0,33.08,31.140,1030.11120
390,2021-05-01 00:26:47,ecdef592,301,United States of America,Bossypants,None,Tina Fey,Tina Fey,5 hrs and 32 mins,Unabridged Audiobook,Arts & Entertainment,4.5,57115.0,30.79,31.140,958.80060
407,2021-05-01 00:28:02,32da6d86,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,31.140,776.94300
519,2021-05-01 00:36:42,c2430abd,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,31.140,776.94300
707,2021-05-01 00:47:22,68b1ea9e,362,United States of America,The Ride of a Lifetime,Lessons Learned from 15 Years as CEO of the Wa...,Robert Iger,Jim Frangione,8 hrs and 45 mins,Unabridged Audiobook,Arts & Entertainment,4.8,12153.0,28.00,31.140,871.92000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997509,2021-07-31 22:34:00,43756c6e,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,32.887,820.53065
1997981,2021-07-31 23:08:18,fe4c7dc3,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,32.887,820.53065
1998041,2021-07-31 23:12:20,fb92ab97,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,32.887,820.53065
1998104,2021-07-31 23:16:13,4da9330b,56,United States of America,Born a Crime,Stories from a South African Childhood,Trevor Noah,Trevor Noah,8 hrs and 44 mins,Unabridged Audiobook,Arts & Entertainment,4.9,182445.0,24.95,32.887,820.53065


In [59]:
# เลือกหนังสือที่อยู่หมวดหมู่ Health & Wellness และ Rating มากกว่า 5
final_df[(final_df['Categories'] == "Health & Wellness" ) | \
         (final_df['THBPrice'] >= 500) ]

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
0,2021-05-01 00:00:01,ad8eca41,1584,Portugal,Sycamore Row,None,John Grisham,Michael Beck,20 hrs and 46 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,25863.0,35.00,31.140,1089.90000
1,2021-05-01 00:00:03,561b26c1,829,United States of America,The 5 Love Languages of Children,The Secret to Loving Children Effectively,Gary Chapman,Chris Fabry,5 hrs and 48 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,2441.0,19.59,31.140,610.03260
2,2021-05-01 00:00:04,81f149e5,1391,Japan,The Guardians,A Novel,John Grisham,Michael Beck,11 hrs and 50 mins,Unabridged Audiobook,"Mystery, Thriller & Suspense",4.6,26781.0,31.50,31.140,980.91000
3,2021-05-01 00:00:07,4f218413,1586,Taiwan,How to Be a 3% Man,None,Corey Wayne,Corey Wayne,7 hrs and 33 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",4.8,11459.0,19.95,31.140,621.24300
4,2021-05-01 00:00:18,a4066781,300,United States of America,Napoleon Hill's Outwitting the Devil,The Secret to Freedom and Success,Napoleon Hill,Dan John Miller,5 hrs and 51 mins,Unabridged Audiobook,Business & Careers,4.8,9305.0,17.49,31.140,544.63860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1998816,2021-07-31 23:59:47,72da1411,600,Portugal,The Time of Contempt,"The Witcher, Book 2",Andrzej Sapkowski,Peter Kenny,11 hrs and 55 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.7,14344.0,25.08,32.887,824.80596
1998817,2021-07-31 23:59:48,620c27c8,462,Montenegro,Endurance,Shackleton's Incredible Voyage,Alfred Lansing,Simon Prebble,10 hrs and 20 mins,Unabridged Audiobook,History,4.7,18764.0,24.95,32.887,820.53065
1998818,2021-07-31 23:59:55,84fe88ab,1656,United States of America,The Spy and the Traitor,The Greatest Espionage Story of the Cold War,Ben Macintyre,John Lee,13 hrs and 20 mins,Unabridged Audiobook,Biographies & Memoirs,4.8,7150.0,31.95,32.887,1050.73965
1998819,2021-07-31 23:59:57,fd6cc4fc,522,United States of America,Lilac Girls,A Novel,Martha Hall Kelly,Cassandra Campbell,17 hrs and 30 mins,Unabridged Audiobook,Literature & Fiction,4.7,19401.0,31.50,32.887,1035.94050


In [60]:
# เลือก Categories หนังสือใน Health & Wellness และ Rating > 7 โดยเรียง Rating จากมากที่สุด
final_df[(final_df['Categories'] == "Health & Wellness" ) | \
         (final_df['Rating'] >= 5) ].sort_values('Rating', ascending=False)

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
356830,2021-05-17 10:18:50,ac516399,2004,Denmark,Quit Smoking Affirmations,Affirmations to Help You Stop Smoking Now and ...,Maxwell Woodland,Ken Kamlet,13 mins,Unabridged Audiobook,Health & Wellness,5.0,44.0,3.95,31.492,124.39340
1449631,2021-07-06 17:19:34,889bb399,723,United States of America,"Harry Potter and the Deathly Hallows, Book 7",None,J.K. Rowling,Jim Dale,21 hrs and 37 mins,Unabridged Audiobook,Children's Audiobooks,5.0,112413.0,44.99,32.271,1451.87229
359055,2021-05-17 12:48:01,ecc7f850,723,United States of America,"Harry Potter and the Deathly Hallows, Book 7",None,J.K. Rowling,Jim Dale,21 hrs and 37 mins,Unabridged Audiobook,Children's Audiobooks,5.0,112413.0,44.99,31.492,1416.82508
359034,2021-05-17 12:46:35,08f8b832,723,United States of America,"Harry Potter and the Deathly Hallows, Book 7",None,J.K. Rowling,Jim Dale,21 hrs and 37 mins,Unabridged Audiobook,Children's Audiobooks,5.0,112413.0,44.99,31.492,1416.82508
605319,2021-05-28 19:57:26,5c486279,838,United States of America,Peaceful on Purpose,"The Power to Remain Calm, Strong, and Confiden...",Joel Osteen,Joel Osteen,2 hrs and 57 mins,Unabridged Audiobook,"Relationships, Parenting & Personal Development",5.0,64.0,28.50,31.257,890.82450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986658,2021-07-31 10:27:11,4ae2773c,1330,Australia,12 Principles for Raising a Child with ADHD,None,Russell A. Barkley PhD,Chris Sorensen,8 hrs,Unabridged Audiobook,Health & Wellness,NaN,NaN,17.49,32.887,575.19363
1986860,2021-07-31 10:40:58,9d393241,1330,France,12 Principles for Raising a Child with ADHD,None,Russell A. Barkley PhD,Chris Sorensen,8 hrs,Unabridged Audiobook,Health & Wellness,NaN,NaN,17.49,32.887,575.19363
1990918,2021-07-31 15:16:42,889ab9e1,2095,Canada,Tree of Secrets,A Tranquil Bedtime Story for Adults,Wysteria T.,Wysteria T.,23 mins,Unabridged Audiobook,Health & Wellness,NaN,NaN,3.95,32.887,129.90365
1992930,2021-07-31 17:29:58,102ff6e8,1330,United Kingdom,12 Principles for Raising a Child with ADHD,None,Russell A. Barkley PhD,Chris Sorensen,8 hrs,Unabridged Audiobook,Health & Wellness,NaN,NaN,17.49,32.887,575.19363


In [61]:
# ค้นหาปีของหนังสือที่ขายในข้อมูลชุดนี้
final_df['timestamp'].groupby(final_df.timestamp.dt.year).agg('count')

timestamp
2021    1998821
Name: timestamp, dtype: int64

In [62]:
# ค้นหาหนังสือที่ขายในประเทศ Australia ในปี 2021
book_australia_2021 = final_df[final_df['country'] == 'Australia' ]
book_australia_2021.head()

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
29,2021-05-01 00:01:41,d9218306,700,Australia,A Game of Thrones,"A Song of Ice and Fire, Book 1",George R.R. Martin,Roy Dotrice,33 hrs and 46 mins,Unabridged Audiobook,Literature & Fiction,4.8,127968.0,56.00,31.14,1743.8400
188,2021-05-01 00:12:46,08526248,297,Australia,Fault Lines,The Social Justice Movement and Evangelicalism...,Voddie T. Baucham,Mirron Willis,8 hrs and 2 mins,Unabridged Audiobook,Religion & Spirituality,4.9,628.0,24.47,31.14,761.9958
192,2021-05-01 00:13:07,3df24951,702,Australia,The Hobbit,None,J. R. R. Tolkien,Rob Inglis,11 hrs and 5 mins,Unabridged Audiobook,Children's Audiobooks,4.7,55068.0,27.99,31.14,871.6086
253,2021-05-01 00:17:22,45efc6b1,1048,Australia,The Ickabog,None,J.K. Rowling,Stephen Fry,7 hrs and 52 mins,Unabridged Audiobook,Children's Audiobooks,4.6,2895.0,24.99,31.14,778.1886
313,2021-05-01 00:22:06,0aeef8fb,646,Australia,Oathbringer,None,Brandon Sanderson,Kate Reading,55 hrs and 5 mins,Unabridged Audiobook,Science Fiction & Fantasy,4.8,48622.0,66.49,31.14,2070.4986


## Query 

ใช้ฟังก์ชัน query ในการค้นหา

In [71]:
# เลือกหนังสือที่อยู่หมวดหมู่ Biographies & Memoirs และเรียง Rating จากมากที่สุด
final_df.query('Categories == "Biographies & Memoirs" ').sort_values('Rating', ascending=False)

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
1442751,2021-07-06 09:35:34,8de6f3d7,579,United States of America,How to Fail at Almost Everything and Still Win...,Kind of the Story of My Life,Scott Adams,Scott Adams,8 hrs and 42 mins,Unabridged Audiobook,Biographies & Memoirs,5.0,13.0,24.50,32.271,790.63950
948585,2021-06-13 14:54:38,13fdf554,1,France,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5.0,47.0,29.65,31.077,921.43305
16675,2021-05-01 18:11:17,855c0771,2206,Canada,Yeager,An Autobiography,Chuck Yeager,Chris Browning,14 hrs and 23 mins,Unabridged Audiobook,Biographies & Memoirs,5.0,521.0,31.50,31.140,980.91000
783420,2021-06-06 00:39:39,dd9f699d,959,United Kingdom,Checking In,How Getting Real About Depression Saved My Lif...,Michelle Williams,Michelle Williams,5 hrs and 20 mins,Unabridged Audiobook,Biographies & Memoirs,5.0,31.0,23.95,31.133,745.63535
788074,2021-06-06 05:56:00,0f6b16d7,1,United States of America,Bamboozled by Jesus,How God Tricked Me into the Life of My Dreams,Yvonne Orji,Yvonne Orji,6 hrs and 31 mins,Unabridged Audiobook,Biographies & Memoirs,5.0,47.0,29.65,31.133,923.09345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1982875,2021-07-31 06:12:03,ac9dfdd8,1282,United Kingdom,Ingmar & Liv's Love Letters,None,Lazaro Droznes,Jennie Litt,1 hr and 24 mins,Unabridged Audiobook,Biographies & Memoirs,NaN,NaN,6.95,32.887,228.56465
1984052,2021-07-31 07:28:23,90bdb0db,1315,Slovakia,The Jigsaw Murders,None,Jeremy Craddock,Jonathan Keeble,10 hrs and 46 mins,Unabridged Audiobook,Biographies & Memoirs,NaN,NaN,18.54,32.887,609.72498
1993475,2021-07-31 18:04:29,005f247d,1188,United States of America,Lapsed,None,Monica Dux,Monica Dux,Not Yet Known,Unabridged Audiobook,Biographies & Memoirs,NaN,NaN,38.17,32.887,1255.29679
1994646,2021-07-31 19:23:10,c98491dc,1228,Argentina,After the Fall,Being American in the World We've Made,Ben Rhodes,Ben Rhodes,12 hrs and 49 mins,Unabridged Audiobook,Biographies & Memoirs,NaN,NaN,31.50,32.887,1035.94050


In [64]:
# query 2 เงือนไข
final_df.query('country == "Japan" & Categories == "Health & Wellness" ').sort_values('Rating', ascending=False)

,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
1200,2021-05-01 01:17:29,0ba9b545,1306,Japan,The Final Emotional Intelligence Bible: 3 Book...,"Everything You Should Know About EQ, Cognitive...",Daniel Anderson,KC Wayman,10 hrs and 35 mins,Unabridged Audiobook,Health & Wellness,5.0,3.0,24.95,31.140,776.94300
1109494,2021-06-21 01:15:47,d03d5d33,2235,Japan,Psychic Empath,A Path to Soul Care for All Highly Sensitive P...,Jason Dyer,Cliff Weldon,3 hrs and 56 mins,Unabridged Audiobook,Health & Wellness,5.0,251.0,14.95,31.617,472.67415
58528,2021-05-03 16:33:06,9bfc7104,2213,Japan,Hypnosis for Weight Loss,The Hypnosis Collection. Rapid Weight Loss for...,Louise Barton,The Voices of Psychology,10 hrs and 4 mins,Unabridged Audiobook,Health & Wellness,5.0,379.0,24.95,31.132,776.74340
1328905,2021-07-01 03:55:05,22b5298e,2233,Japan,Emotional Intelligence: 21 Proven Ways to Impr...,"Emotional Intelligence 2.0, Book 1",Eric Skinner,Sam Slydell,3 hrs and 24 mins,Unabridged Audiobook,Health & Wellness,5.0,252.0,14.95,32.073,479.49135
1743745,2021-07-20 06:06:20,00e6586f,1836,Japan,1001 Names for Girls,Meanings and Origins for the Most Popular Baby...,Wynne Foster,Nathan Levasseur,3 hrs and 7 mins,Unabridged Audiobook,Health & Wellness,5.0,21.0,14.95,32.829,490.79355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1342684,2021-07-01 19:23:29,aac3607b,1267,Japan,The Stress-Proof Brain,Master Your Emotional Response to Stress Using...,Abhishek Subidi,Derik Hendriksen,3 hrs and 46 mins,Unabridged Audiobook,Health & Wellness,NaN,NaN,14.95,32.073,479.49135
1578600,2021-07-12 16:06:41,fee2c7eb,1267,Japan,The Stress-Proof Brain,Master Your Emotional Response to Stress Using...,Abhishek Subidi,Derik Hendriksen,3 hrs and 46 mins,Unabridged Audiobook,Health & Wellness,NaN,NaN,14.95,32.664,488.32680
1791095,2021-07-22 10:21:37,71bd4641,1231,Japan,Learn like a Pro,Science-Based Tools to Become Better at Anything,Barbara Oakley PhD,Robert Petkoff,3 hrs and 38 mins,Unabridged Audiobook,Health & Wellness,NaN,NaN,10.39,32.887,341.69593
1905478,2021-07-27 17:13:32,eac87cb7,1231,Japan,Learn like a Pro,Science-Based Tools to Become Better at Anything,Barbara Oakley PhD,Robert Petkoff,3 hrs and 38 mins,Unabridged Audiobook,Health & Wellness,NaN,NaN,10.39,32.937,342.21543


In [65]:
# หาหนังสือผู้เขียน Neil deGrasse Tyson
final_df.query('Book_Author == "Neil deGrasse Tyson" ')


,timestamp,user_id,book_id,country,Book_Title,Book_Subtitle,Book_Author,Book_Narrator,Audio_Runtime,Audiobook_Type,Categories,Rating,Total_No_of_Ratings,Price,conversion_rate,THBPrice
123,2021-05-01 00:08:18,b2a5473b,568,United States of America,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,31.140,544.01580
145,2021-05-01 00:10:04,533dce6f,568,Japan,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,31.140,544.01580
308,2021-05-01 00:21:45,395c21f0,568,Japan,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,31.140,544.01580
346,2021-05-01 00:24:13,9795de47,568,United Kingdom,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,31.140,544.01580
720,2021-05-01 00:47:58,f63823bd,568,United States of America,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,31.140,544.01580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997130,2021-07-31 22:08:37,e1366419,568,Finland,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,32.887,574.53589
1997768,2021-07-31 22:52:56,7cec6dd4,568,Algeria,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,32.887,574.53589
1997840,2021-07-31 22:57:20,e016a2a5,568,Israel,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,32.887,574.53589
1998367,2021-07-31 23:31:22,b109b66d,568,Norway,Astrophysics for People in a Hurry,None,Neil deGrasse Tyson,Neil deGrasse Tyson,3 hrs and 41 mins,Unabridged Audiobook,Science & Engineering,4.6,33843.0,17.47,32.887,574.53589


##Summary 

หา Summary เฉพาะคอลัมน์ Categories 

In [73]:
# เลือกคอลัมน์ Categories และ THBPrice
df_THBPrice = final_df[['Categories','THBPrice']]

# ทำการหาค่า Summary หนังสือแต่ละ category
summary_Categories = df_THBPrice.groupby(['Categories'])
summary_Categories.describe()

THBPrice               \
                                                    count         mean   
Categories                                                               
Arts & Entertainment                              38918.0   826.525182   
Biographies & Memoirs                            197481.0   965.123838   
Business & Careers                               156190.0   788.932805   
Children's Audiobooks                            141040.0  1047.648073   
Computers & Technology                             1121.0   610.973649   
Education & Learning                               5140.0   381.732657   
Erotica                                            5015.0   556.210681   
Health & Wellness                                 94204.0   869.873452   
History                                           33314.0   963.904823   
Home & Garden                                      4398.0   759.056422   
LGBTQ+                                             5911.0   769.277559   
Literature & Fiction                             547364.0  1011.028235   
Money & Finance                                   27180.0   837.585216   
Mystery, Thriller & Suspense                     111706.0   961.336575   
Politics & Social Sciences                        33767.0   881.718990   
Relationships, Parenting & Personal Development  115570.0   737.129823   
Religion & Spirituality                           11724.0   774.650707   
Romance                                           56693.0  1208.390272   
Science & Engineering                              8425.0   623.857938   
Science Fiction & Fantasy                        326089.0  1279.566872   
Sports & Outdoors                                    49.0  1109.005194   
Teen                                              54658.0  1028.123707   
Travel & Tourism                                    585.0   890.123015   

                                                                         \
                                                        std         min   
Categories                                                                
Arts & Entertainment                             117.399691   122.58035   
Biographies & Memoirs                            238.888219    61.75567   
Business & Careers                               191.723970    86.58207   
Children's Audiobooks                            346.184722     0.00000   
Computers & Technology                           250.447444   122.58035   
Education & Learning                             352.723600     0.00000   
Erotica                                          294.622202   122.58035   
Health & Wellness                                222.561822   122.58035   
History                                          304.021493     0.00000   
Home & Garden                                    134.136010   122.58035   
LGBTQ+                                           124.302265     0.00000   
Literature & Fiction                             290.810338     0.00000   
Money & Finance                                  187.267415   122.58035   
Mystery, Thriller & Suspense                     226.506204     0.00000   
Politics & Social Sciences                       230.158080   108.30517   
Relationships, Parenting & Personal Development  184.398875     0.00000   
Religion & Spirituality                          310.716446    86.58207   
Romance                                          271.216476   215.67935   
Science & Engineering                            184.308670   122.58035   
Science Fiction & Fantasy                        473.502411     0.00000   
Sports & Outdoors                                 22.845219  1085.84467   
Teen                                             257.966850     0.00000   
Travel & Tourism                                  18.948305   868.92400   

                                                                         \
                                                        25%         50%   
Categories             